# RNN으로_텍스트_분석하기
* RNN: 순환신경망
* 언어, 시계열 데이터 분석에 주로 사용
* 길이가 길어지면 기울기 소실 문제  방생
* LSTM, GRU 

In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding
from tensorflow.keras.utils import to_categorical
import pandas as pd
import numpy as np

2025-05-21 15:11:17.378266: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-21 15:11:17.539518: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-21 15:11:17.543816: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-05-21 15:11:17.543839: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

# 텐서플로에서 텍스트 전처리 하기
* 토큰화 : 문장을 단어 혹은 형태소로 쪼개는 것
* 원핫인코딩 : 문자를 벡터화
* 임베딩(enbedding):백터화 -> 원핫인코딩을 더 축소

In [7]:
text = "해보지 않으면 해낼수 없다"

In [9]:
result = text_to_word_sequence(text)
result

['해보지', '않으면', '해낼수', '없다']

단어 빈도수 세기

In [5]:
docs = ['먼저 텍스트의 각 단어를 나누어 토큰화 합니다',
       '텍스트의 단어로 토큰화해야 딥러닝에서 인식됩니다',
       '토큰화한 결과는 딥러닝에서 사용할 수 있습니다',
       '텍스트 전처리에는 벡터화 원핫인코딩 패딩으로 길이 맞추기 등이 필요합니다',
       '딥러닝 쉽지 않네요']

In [12]:
token = Tokenizer()
token.fit_on_texts(docs)
print("단어카운트 :", token.word_counts, end="\n\n")
print("문장카운트 :", token.document_count, end="\n\n")
print('각 단어가 몇 개의 문장에 포함되어있는지 계산 :', token.word_docs, end="\n\n")
print("각 단어의 매겨진 인덱스값: ", token.word_index, end="\n\n")

단어카운트 : OrderedDict([('먼저', 1), ('텍스트의', 2), ('각', 1), ('단어를', 1), ('나누어', 1), ('토큰화', 1), ('합니다', 1), ('단어로', 1), ('토큰화해야', 1), ('딥러닝에서', 2), ('인식됩니다', 1), ('토큰화한', 1), ('결과는', 1), ('사용할', 1), ('수', 1), ('있습니다', 1), ('텍스트', 1), ('전처리에는', 1), ('벡터화', 1), ('원핫인코딩', 1), ('패딩으로', 1), ('길이', 1), ('맞추기', 1), ('등이', 1), ('필요합니다', 1), ('딥러닝', 1), ('쉽지', 1), ('않네요', 1)])

문장카운트 : 5

각 단어가 몇 개의 문장에 포함되어있는지 계산 : defaultdict(<class 'int'>, {'합니다': 1, '단어를': 1, '토큰화': 1, '텍스트의': 2, '먼저': 1, '각': 1, '나누어': 1, '딥러닝에서': 2, '단어로': 1, '토큰화해야': 1, '인식됩니다': 1, '수': 1, '있습니다': 1, '결과는': 1, '토큰화한': 1, '사용할': 1, '전처리에는': 1, '등이': 1, '원핫인코딩': 1, '패딩으로': 1, '텍스트': 1, '길이': 1, '맞추기': 1, '필요합니다': 1, '벡터화': 1, '않네요': 1, '쉽지': 1, '딥러닝': 1})

각 단어의 매겨진 인덱스값:  {'텍스트의': 1, '딥러닝에서': 2, '먼저': 3, '각': 4, '단어를': 5, '나누어': 6, '토큰화': 7, '합니다': 8, '단어로': 9, '토큰화해야': 10, '인식됩니다': 11, '토큰화한': 12, '결과는': 13, '사용할': 14, '수': 15, '있습니다': 16, '텍스트': 17, '전처리에는': 18, '벡터화': 19, '원핫인코딩': 20, '패딩으로': 21, '길이': 22, '맞추기': 23, '등이': 

In [ ]:
'딥러닝 쉽지 않네요' -> [26, 27, 28]

In [13]:
x = token.texts_to_sequences(docs)
print(x)

[[3, 1, 4, 5, 6, 7, 8], [1, 9, 10, 2, 11], [12, 13, 2, 14, 15, 16], [17, 18, 19, 20, 21, 22, 23, 24, 25], [26, 27, 28]]


가장 긴 백터의 길이 추출

In [15]:
max([len(i) for i in x])

9

# 가장 긴 길이의 문장에 맞추어 패딩
* 가장 긴 문장 길이 + 1의 길이로 패딩
* 문장의 시작에는 반드시 0이 있어야 함

In [16]:
padded_x = pad_sequences(x, max([len(i) for i in x])+1)
padded_x

array([[ 0,  0,  0,  3,  1,  4,  5,  6,  7,  8],
       [ 0,  0,  0,  0,  0,  1,  9, 10,  2, 11],
       [ 0,  0,  0,  0, 12, 13,  2, 14, 15, 16],
       [ 0, 17, 18, 19, 20, 21, 22, 23, 24, 25],
       [ 0,  0,  0,  0,  0,  0,  0, 26, 27, 28]], dtype=int32)

In [17]:
padded_x.shape

(5, 10)

# 텍스트를 읽고 긍정, 부정 예측하기

In [18]:
docs2 = ['너무 재미있네요',
        '최고예요',
        '참 신기한 딥러닝이네요',
        '인공지능 칭찬합니다',
        '더 자세히 배우고 싶어요',
        '변화가 너무 빨라요',
        "GPT성능이 생각보다 별로네요",
        '제미나이보다는 낫죠',
        '나는 차라리 라마를 쓴다',
        '유료 결제 싫어요']


In [19]:
classes = np.array([1,1,1,1,1,0,0,1,0,0])

In [25]:
token = Tokenizer()
token.fit_on_texts(docs2)

In [33]:
x = token.texts_to_sequences(docs2)
print(x)

[[1, 2], [3], [4, 5, 6], [7, 8], [9, 10, 11, 12], [13, 1, 14], [15, 16, 17], [18, 19], [20, 21, 22, 23], [24, 25, 26]]


In [34]:
padding_x = pad_sequences(x, max([len(i) for i in x])+1)
padding_x

array([[ 0,  0,  0,  1,  2],
       [ 0,  0,  0,  0,  3],
       [ 0,  0,  4,  5,  6],
       [ 0,  0,  0,  7,  8],
       [ 0,  9, 10, 11, 12],
       [ 0,  0, 13,  1, 14],
       [ 0,  0, 15, 16, 17],
       [ 0,  0,  0, 18, 19],
       [ 0, 20, 21, 22, 23],
       [ 0,  0, 24, 25, 26]], dtype=int32)

In [35]:
word_size = len(token.word_index) +1
word_size

27

In [36]:
Embedding(word_size, 8)

In [ ]:
for i in 

In [38]:
model = Sequential()
model.add(Embedding(word_size, 8))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 8)           216       
                                                                 
 dense (Dense)               (None, None, 1)           9         
                                                                 
Total params: 225
Trainable params: 225
Non-trainable params: 0
_________________________________________________________________


In [40]:
model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])
model.fit(padding_x, classes, epochs=20)
print(model.evaluate(padding_x, classes))

Epoch 1/20


ValueError: in user code:

    File "/home/jihyun/miniforge3/envs/face/lib/python3.10/site-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/home/jihyun/miniforge3/envs/face/lib/python3.10/site-packages/keras/engine/training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/jihyun/miniforge3/envs/face/lib/python3.10/site-packages/keras/engine/training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "/home/jihyun/miniforge3/envs/face/lib/python3.10/site-packages/keras/engine/training.py", line 994, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/jihyun/miniforge3/envs/face/lib/python3.10/site-packages/keras/engine/training.py", line 1052, in compute_loss
        return self.compiled_loss(
    File "/home/jihyun/miniforge3/envs/face/lib/python3.10/site-packages/ker